In [ ]:
import tkinter as tk
from tkinter import messagebox
import sqlite3
import datetime

# -------- SIMPLE SENTIMENT LOGIC --------
positive_words = ["good", "great", "excellent", "amazing", "fast", "happy", "love",
                  "satisfied", "nice", "best", "awesome", "recommend", "delighted"]
negative_words = ["bad", "poor", "slow", "terrible", "sad", "hate", "worst",
                  "unsatisfied", "disappointed", "problem", "issue", "late", "angry"]

def analyze_sentiment(text):
    text = text.lower()
    pos = sum(w in text for w in positive_words)
    neg = sum(w in text for w in negative_words)

    score = pos - neg
    if score > 0:
        sentiment = "Positive 😁 "
    elif score < 0:
        sentiment = "Negative 😡 "
    else:
        sentiment = "Neutral 🙂"

    return pos, neg, sentiment

# -------- DATABASE SETUP --------
conn = sqlite3.connect("sentiment.db")
cur = conn.cursor()

cur.execute("""
CREATE TABLE IF NOT EXISTS feedbacks (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    feedback TEXT,
    sentiment TEXT,
    pos_count INTEGER,
    neg_count INTEGER,
    created_at TEXT
)
""")

conn.commit()

def save_to_db(feedback, sentiment, pos, neg):
    time_now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    cur.execute("INSERT INTO feedbacks VALUES (NULL, ?, ?, ?, ?, ?)",
                (feedback, sentiment, pos, neg, time_now))
    conn.commit()

# ----------- UI PART -----------
root = tk.Tk()
root.title("Simple Sentiment Analyzer")
root.geometry("500x400")

tk.Label(root, text="Enter Feedback:", font=("Arial", 12)).pack()

txt = tk.Text(root, height=8, width=50)
txt.pack()

result_label = tk.Label(root, text="", font=("Arial", 14, "bold"))
result_label.pack(pady=10)

def analyze_and_save():
    feedback = txt.get("1.0", tk.END).strip()

    if feedback == "":
        messagebox.showwarning("Empty", "Please enter some feedback!")
        return

    pos, neg, sentiment = analyze_sentiment(feedback)

    result_label.config(text=f"Sentiment: {sentiment}")

    save_to_db(feedback, sentiment, pos, neg)
    messagebox.showinfo("Saved", "Feedback analyzed and saved to database!")

tk.Button(root, text="Analyze", command=analyze_and_save, font=("Arial", 12),
          bg="lightblue").pack(pady=10)

root.mainloop()